In [5]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
score=0
model = SentenceTransformer('bert-base-nli-mean-tokens')
questionEcoding = model.encode("Do you think it is wonderful to be alive now?")
print(questionEcoding)
questionEcoding.reshape(-1,1)
questionEcoding.shape

[ 9.79031861e-01 -5.75995803e-01  1.56253839e+00  6.09389096e-02
  2.97991633e-01 -1.94869354e-01 -1.77555293e-01  5.51897168e-01
  1.87673599e-01 -6.15448594e-01 -2.94246674e-01  1.66284665e-01
  5.07154524e-01  4.34541404e-01 -4.78883162e-02  1.72117814e-01
 -4.04785015e-03 -3.06996554e-01  1.59383669e-01 -7.55243361e-01
 -5.11771798e-01 -3.26969683e-01 -2.22894683e-01 -2.60953903e-01
  1.61356851e-01  4.23119515e-02  1.25137135e-01 -7.75759637e-01
 -3.31258886e-02  2.88941652e-01 -5.09354711e-01  5.82666457e-01
  2.44585142e-01 -3.27443630e-01  3.58069167e-02  1.35804880e+00
 -2.06866246e-02 -1.61645830e-01 -2.12310404e-01 -5.01447439e-01
  8.67741823e-01 -7.19138801e-01  2.96538711e-01  4.81900007e-01
 -9.27242577e-01 -3.63863766e-01 -7.32846141e-01  3.52079362e-01
  5.20612776e-01 -6.59266189e-02  3.73048410e-02 -9.11236405e-01
 -5.70386350e-01  1.66083410e-01 -7.94305384e-01  3.04329187e-01
  2.10129768e-01 -7.65625834e-01 -1.83171347e-01  3.52174602e-02
 -2.69719958e-01 -5.65153

(768,)

In [6]:
answerEcoding = model.encode("Life is beautiful")


In [7]:
sim_arr=cosine_similarity([questionEcoding],[answerEcoding])
print('sim_arr:',sim_arr)
if(sim_arr>0.5):
    score=score+1
print('score:',score)
  #finalScore = (score/(len(answers)/2))*100

sim_arr: [[0.77799284]]
score: 1


In [8]:
from torch.nn.functional import softmax
from transformers import BertForNextSentencePrediction, BertTokenizer






In [14]:
seq_A = 'Can you remember all the names of your sons and daughters?'
seq_B = 'I eat banana'

In [9]:
# load pretrained model and a pretrained tokenizer
model = BertForNextSentencePrediction.from_pretrained('bert-base-cased')
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

# encode the two sequences. Particularly, make clear that they must be 
# encoded as "one" input to the model by using 'seq_B' as the 'text_pair'


Downloading: 100%|██████████| 570/570 [00:00<00:00, 383kB/s]
Downloading: 100%|██████████| 436M/436M [00:46<00:00, 9.45MB/s] 
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForNextSentencePrediction: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Downloading: 100%|██████████| 2

{'input_ids': tensor([[  101,   146,  1176, 18621,   106,   102,  2091,  1128,  1176,  1172,
           136,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [15]:
encoded = tokenizer.encode_plus(seq_A, text_pair=seq_B, return_tensors='pt')
print(encoded)
# {'input_ids': tensor([[  101,   146,  1176, 18621,   106,   102,  2091,  1128,  1176,  1172, 136,   102]]),
#  'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]]),
#  'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
# NOTE how the token_type_ids are 0 for all tokens in seq_A and 1 for seq_B, 
# this way the model knows which token belongs to which sequence

# a model's output is a tuple, we only need the output tensor containing
# the relationships which is the first item in the tuple

{'input_ids': tensor([[  101,  2825,  1128,  2676,  1155,  1103,  2666,  1104,  1240,  3824,
          1105,  5421,   136,   102,   146,  3940, 21806,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [16]:
seq_relationship_logits = model(**encoded)[0]

# we still need softmax to convert the logits into probabilities
# index 0: sequence B is a continuation of sequence A
# index 1: sequence B is a random sequence
probs = softmax(seq_relationship_logits, dim=1)

print(seq_relationship_logits)
print(probs)

tensor([[-0.3373,  1.4242]], grad_fn=<AddmmBackward0>)
tensor([[0.1466, 0.8534]], grad_fn=<SoftmaxBackward0>)


In [18]:
print(probs.values.__get__)

AttributeError: 'builtin_function_or_method' object has no attribute '__get__'